MNIST を Swift for TensorFlow で計算しました

- モデルは [公式サンプル](https://github.com/tensorflow/swift-models/blob/stable/MNIST/MNIST.swift) そのままです
- MNISTのデータは colab のサンプルデータを利用した
  - ```sample_data/mnist_train_small.csv``` 
  - ```sample_data/mnist_test.csv``` 


In [1]:
import Foundation
import TensorFlow

/// サンプルデータの読み込み
class MNIST{
  
  var xTrain: Tensor<Float>!
  var xTest: Tensor<Float>!
  var yTrain: Tensor<Int32>!
  var yTest: Tensor<Int32>!
  
  init(){
    self.read()
  }
  
  
  func readCsv(filePath: String) ->  (Tensor<Float>,   Tensor<Int32>){
    print("read csv \(filePath)")
    
    let url = URL(fileURLWithPath: filePath)
    var x:  [Float] = []
    var y:  [Int32] =  []
      
    do{
      let str = try String(contentsOf: url, encoding:String.Encoding.utf8)
      let lines = str.components(separatedBy: .newlines)
      
      for line in lines{
        let arr = line.components(separatedBy: ",")    
        if arr.count <= 1{
          continue
        }
        y.append(Int32(arr[0])! )
        
        let range = 1 ..< arr.count
        let temps:[Float] = range.map {  Float(arr[$0])! }
        x.append(contentsOf: temps)
      }
    }catch {
      print("error \(error)")
    }
    
    let row = Int32(y.count)
    let imageHeight: Int32 = 28
    let imageWidth: Int32 = 28
    
    let tx =  Tensor(shape: [row, 1, imageWidth, imageHeight], scalars: x).transposed(withPermutations: [0, 2, 3, 1]) / 255
    let ty:  Tensor<Int32> =  Tensor(y)
     
    return (tx, ty)
  }
  
  func read(){
  
    let testFile = "sample_data/mnist_test.csv"
    let trainFile = "sample_data/mnist_train_small.csv"
      
    let (xTrain, yTrain) = self.readCsv(filePath: trainFile)
    let (xTest, yTest) = self.readCsv(filePath: testFile)
  
    self.xTrain = xTrain
    self.yTrain = yTrain
    self.xTest = xTest
    self.yTest = yTest
  }
  
}

let mnist = MNIST()


read csv sample_data/mnist_train_small.csv
read csv sample_data/mnist_test.csv


In [2]:
// https://github.com/tensorflow/swift-models/blob/stable/MNIST/MNIST.swift

/// A classifier.
struct Classifier: Layer {
    var conv1a = Conv2D<Float>(filterShape: (3, 3, 1, 32), activation: relu)
    var conv1b = Conv2D<Float>(filterShape: (3, 3, 32, 64), activation: relu)
    var pool1 = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))

    var dropout1a = Dropout<Float>(probability: 0.25)
    var flatten = Flatten<Float>()
    var layer1a = Dense<Float>(inputSize: 9216, outputSize: 128, activation: relu)
    var dropout1b = Dropout<Float>(probability: 0.5)
    var layer1b = Dense<Float>(inputSize: 128, outputSize: 10, activation: softmax)

    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        var tmp = input
        tmp = conv1a.applied(to: tmp, in: context)
        tmp = conv1b.applied(to: tmp, in: context)
        tmp = pool1.applied(to: tmp, in: context)
        tmp = dropout1a.applied(to: tmp, in: context)
        tmp = flatten.applied(to: tmp, in: context)
        tmp = layer1a.applied(to: tmp, in: context)
        tmp = dropout1b.applied(to: tmp, in: context)
        tmp = layer1b.applied(to: tmp, in: context)
        return tmp
    }
}

let epochCount = 12
let batchSize = 100

func minibatch<Scalar>(in x: Tensor<Scalar>, at index: Int) -> Tensor<Scalar> {
    let start = Int32(index * batchSize)
    return x[start..<start+Int32(batchSize)]
}

// let (images, numericLabels) = readMNIST(imagesFile: "train-images-idx3-ubyte",
//                                         labelsFile: "train-labels-idx1-ubyte")

let images = mnist.xTrain!
let numericLabels = mnist.yTrain!
let labels = Tensor<Float>(oneHotAtIndices: numericLabels, depth: 10)

var classifier = Classifier()
let context = Context(learningPhase: .training)
let optimizer = RMSProp<Classifier, Float>()

// The training loop.
for epoch in 0..<epochCount {
  
  let date = Date()
  
  
    var correctGuessCount = 0
    var totalGuessCount = 0
    var totalLoss: Float = 0
    for i in 0 ..< Int(labels.shape[0]) / batchSize {
        let x = minibatch(in: images, at: i)
        let y = minibatch(in: numericLabels, at: i)
        // Compute the gradient with respect to the model.
        let 𝛁model = classifier.gradient { classifier -> Tensor<Float> in
            let ŷ = classifier.applied(to: x, in: context)
            let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== y
            correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
            totalGuessCount += batchSize
            let loss = softmaxCrossEntropy(logits: ŷ, labels: y)
            totalLoss += loss.scalarized()
            return loss
        }
        // Update the model's differentiable variables along the gradient vector.
        optimizer.update(&classifier.allDifferentiableVariables, along: 𝛁model)
    }
  
  let diff = Date().timeIntervalSince(date)
    
    let accuracy = Float(correctGuessCount) / Float(totalGuessCount)
    print("""
          [Epoch \(epoch)] \
          Loss: \(totalLoss), \
          Accuracy: \(correctGuessCount)/\(totalGuessCount) (\(accuracy)), \
          elapsed_time: \(diff) sec
          """)
}



[Epoch 0] Loss: 329.09583, Accuracy: 16471/20000 (0.82355), elapsed_time: 71.8861780166626 sec
[Epoch 1] Loss: 305.26334, Accuracy: 18750/20000 (0.9375), elapsed_time: 71.62785196304321 sec
[Epoch 2] Loss: 301.8193, Accuracy: 19079/20000 (0.95395), elapsed_time: 70.78193807601929 sec
[Epoch 3] Loss: 300.4459, Accuracy: 19210/20000 (0.9605), elapsed_time: 70.57480204105377 sec
[Epoch 4] Loss: 299.07968, Accuracy: 19332/20000 (0.9666), elapsed_time: 70.65001308917999 sec
[Epoch 5] Loss: 298.65607, Accuracy: 19376/20000 (0.9688), elapsed_time: 71.37785696983337 sec
[Epoch 6] Loss: 298.01047, Accuracy: 19438/20000 (0.9719), elapsed_time: 70.87592697143555 sec
[Epoch 7] Loss: 297.48526, Accuracy: 19494/20000 (0.9747), elapsed_time: 70.78634297847748 sec
[Epoch 8] Loss: 297.0606, Accuracy: 19527/20000 (0.97635), elapsed_time: 70.71874690055847 sec
[Epoch 9] Loss: 296.8288, Accuracy: 19552/20000 (0.9776), elapsed_time: 71.18163001537323 sec
[Epoch 10] Loss: 296.45755, Accuracy: 19589/20000 (0

In [3]:
// テストデータで評価

let testImages = mnist.xTest!
let testNumericLabels = mnist.yTest!

let testY = classifier.inferring(from: testImages)
//print("testY: \(testY)")

let testCorrectPredictions = testY.argmax(squeezingAxis: 1)  .== testNumericLabels
let testCorrectGuessCount = Int(Tensor<Int32>(testCorrectPredictions).sum().scalarized())
let acc = Float(testCorrectGuessCount) / Float(testNumericLabels.shape[0])

print("testCorrectGuessCount: \(testCorrectGuessCount)/\(testNumericLabels.shape[0]), acc: \(acc)")

tcmalloc: large alloc 1474560000 bytes == 0x42698000 @  0x7f54cd936b6b 0x7f54cd956379 0x7f54b458f6c7 0x7f54b41d2242 0x7f54b428873b 0x7f54b425937e 0x7f54b42597b9 0x7f54b425989a 0x7f54bbc44ed5 0x7f54b7f3fc9f 0x7f54b7f3e9fd 0x7f54b7ee2041 0x7f54b7ee6cb5 0x7f54b7ee8f55 0x7f54b58689a7 0x7f54a12f7003 0x7f54a12f6cf9 0x7f54a13229f6 0x7f54a1311e19 0x7f54a13d9010 0x7f54cdd3cb66 0x7f54cdd401af 0x7f54a13d3e22 0x7f54cdd3045e 0x400650 0x7f54cdd3c3d0
tcmalloc: large alloc 6635634688 bytes == 0x9a4d8000 @  0x7f54cd9541e7 0x7f54bb8c63b5 0x7f54bb95dc8e 0x7f54bbc05ad0 0x7f54bbc43ca4 0x7f54bbc440db 0x7f54bbc44fc8 0x7f54b7f3fc9f 0x7f54b7f3e9fd 0x7f54b7ee2041 0x7f54b7ee6cb5 0x7f54b7ee8f55 0x7f54b58689a7 0x7f54a12f7003 0x7f54a12f6cf9 0x7f54a13229f6 0x7f54a1311e19 0x7f54a13d9010 0x7f54cdd3cb66 0x7f54cdd401af 0x7f54a13d3e22 0x7f54cdd3045e 0x400650 0x7f54cdd3c3d0
tcmalloc: large alloc 1474560000 bytes == 0x9a4d8000 @  0x7f54cd936b6b 0x7f54cd956379 0x7f54b458f6c7 0x7f54b41d2242 0x7f54b428873b 0x7f54b425937e 0x7f